In [1]:
# Tensorflow / Keras
from tensorflow import keras # for building Neural Networks
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Dense # for creating regular densely-connected NN layers.
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

# Data manipulation
import pandas as pd # for data manipulation
import numpy as np # for data manipulation

# Sklearn
import sklearn # for model evaluation
from sklearn.model_selection import train_test_split # for splitting data into train and test samples
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Erstellung eigener Aktivierungsfunktion
from keras import backend as K
import os
import scipy
from sklearn.mixture import GaussianMixture

# Tensorflow / Keras
from tensorflow import keras # for building Neural Networks
from keras.models import Sequential, load_model # add load_model to the import

In [3]:
class gmm_model():

    def __init__(self):
        self.test_size = 0.35
        self.p = 0.3

    def import_data(self, sample_size = 11500):
        csv_files = []
        for filename in os.listdir():
            if filename == "Datasets":
                for csv_file in os.listdir(filename):
                    if csv_file.endswith('.csv'):
                        csv_files.append(csv_file)
        raw_data = pd.read_csv(os.path.join("Datasets", csv_files[1]))
        raw_data.drop(["column_a"], axis=1, inplace=True)
        
        # Auswahl von 4000 zufälligen Zeilen
        raw_data = raw_data.sample(n=sample_size, random_state=42)

        
        Y = raw_data["y"].astype('float32') - 1
        X = raw_data.drop(["y"], axis=1).astype('float32')

        return X, Y, raw_data

    def gen_gmm(self, data):
        data.drop(["y"], axis=1, inplace=True)
        # erstellen GMM über gesamtes
        gmm = GaussianMixture(n_components=3, covariance_type="diag")
        gmm.fit(data)
        return gmm

    def gen_miss_values(self, data, p):
        shape = data.shape
        new_df = data.copy().astype(np.float64)
        missing = np.random.binomial(1, self.p, shape)
        data[missing.astype('bool')] = np.nan

        return data

    def train_test_split(self, X, Y, test_size):
        # train_test_split
        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=self.test_size, random_state=42)
        return X_train, X_test, y_train, y_test

    def gmm_activation_smieja(self, data_missing, gmm, model):
        w = model.layers[0].get_weights()[0]  # Gewichte von erster Schicht
        b = model.layers[0].get_weights()[1][:, np.newaxis]  # Bias erste Schicht
        output_list = []
        # man muss nur durch spalten iterieren
        for s in range(0, len(pd.DataFrame(w).columns)):
            for i in range(0, len(data_missing)):  # iteriere DF mit NaN-Values
                list_nan = np.array(data_missing.iloc[i])

                result = 0
                for ii in range(0, gmm.n_components):  # iteriere GMM mit n_komp
                    list_gmm_cov = gmm.covariances_[ii]
                    list_gmm_mean = gmm.means_[ii]
                    p_i = gmm.weights_[ii]

                    # Erstellen von GMM-Vektoren
                    # create cov_gmm: in gmm_covariances_vektor werte 0 setzen die in der jeweiligen Spalte von data_missing nicht NaN sind
                    global s_i
                    s_i = []
                    for y in range(len(list_gmm_cov)):
                        if np.isnan(list_nan[y]):
                            s_i.append(list_gmm_cov[y])
                        else:
                            s_i.append(0)

                    # create mean_gmm: in gmm_mean_vektor werden alle festen werte aufgenommen aus data_missing und NaN-Werte durch list_gmm_mean ersetzt
                    m_i = []
                    for y in range(len(list_gmm_mean)):
                        if not np.isnan(list_nan[y]):
                            m_i.append(list_nan[y])
                        else:
                            m_i.append(list_gmm_mean[y])

                    # Berechnen ReLUw,b(F)
                    ft = p_i * np.sqrt(np.transpose(np.array(w[:, s]) * w[:, s]) @ s_i)
                    z = np.transpose(np.array(w[:, s])) @ m_i + b[s]  # Zähler für NR w
                    n = np.sqrt(np.transpose(np.array(w[:, s]) * w[:, s]) @ s_i)  # Nenner für NR w
                    nr_w = z / n  # Input in NR
                    nr = 1 / np.sqrt(2 * np.pi) * np.exp(-(np.square(nr_w) / 2)) + (nr_w / 2) * (
                                1 + scipy.special.erf(nr_w / np.sqrt(2)))  # NR-Funktion
                    ReLUwb = ft * nr  # Summe aus i-GMM-Komponenten
                    # print(f'ReLUwb:{ReLUwb:} ft:{ft:} z:{z:} n:{n:} nr_w:{nr_w:} nr:{nr:}')
                    result += ReLUwb
                    # print(f'resutl{result}')
            output_list.append(result)
        return output_list

    def model_01(self, X_train, X_test, y_train, y_test):
        model = tf.keras.Sequential(name="Model")
        model.add(tf.keras.layers.Dense(units=32, activation=tf.nn.relu, input_shape=[len(X_train.columns)]))
        model.add(tf.keras.layers.Dense(units=64, activation=tf.nn.relu))
        model.add(tf.keras.layers.Dense(units=128, activation=tf.nn.relu))
        model.add(tf.keras.layers.Dense(units=5, activation=tf.nn.softmax))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        early_stop = EarlyStopping(monitor='val_loss', patience=5)

        # Fit model with early stopping
        model.fit(X_train, tf.keras.utils.to_categorical(y_train), epochs=50,
                  validation_data=(X_test, tf.keras.utils.to_categorical(y_test)),
                  callbacks=[early_stop])
        return model

    def model_02(self, X_train, X_test, y_train, y_test):
        model = tf.keras.Sequential(name="Model")
        model.add(tf.keras.layers.Dense(units=32, activation=tf.nn.relu, input_shape=[len(X_train.columns)]))
        model.add(tf.keras.layers.Dense(units=64, activation=tf.nn.relu))
        model.add(tf.keras.layers.Dense(units=128, activation=tf.nn.relu))
        model.add(tf.keras.layers.Dense(units=256, activation=tf.nn.relu))
        model.add(tf.keras.layers.Dense(units=5, activation=tf.nn.softmax))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        early_stop = EarlyStopping(monitor='val_loss', patience=5)

        # Fit model with early stopping
        model.fit(X_train, tf.keras.utils.to_categorical(y_train), epochs=50,
                  validation_data=(X_test, tf.keras.utils.to_categorical(y_test)),
                  callbacks=[early_stop])
        return model

    def model_03(self, X_train, X_test, y_train, y_test):
        model = tf.keras.Sequential(name="Model")
        model.add(tf.keras.layers.Dense(units=100, activation=tf.nn.relu, input_shape=[len(X_train.columns)]))
        model.add(tf.keras.layers.Dense(units=400, activation=tf.nn.relu))
        model.add(tf.keras.layers.Dense(units=600, activation=tf.nn.relu))
        model.add(tf.keras.layers.Dense(units=5, activation=tf.nn.softmax))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        early_stop = EarlyStopping(monitor='val_loss', patience=5)

        # Fit model with early stopping
        model.fit(X_train, tf.keras.utils.to_categorical(y_train), epochs=50,
                  validation_data=(X_test, tf.keras.utils.to_categorical(y_test)),
                  callbacks=[early_stop])
        return model

    def predict_model(self, X_test_m, model):
        if len(model.layers) == 4:
            predictions = []
            for index, row in X_test_m.iterrows():
                row_df = pd.DataFrame([row])
                row_df.columns = [f"x{i + 1}" for i in range(len(row_df.columns))]

                gmm_output = self.gmm_activation_smieja(row_df, gmm, model)

                input_data = np.array(gmm_output).reshape((1, len(gmm_output)))
                get_layer_output = K.function([model.layers[1].input], [model.layers[1].output])
                output = get_layer_output([input_data])[0]
                get_layer_output = K.function([model.layers[2].input], [model.layers[2].output])
                output = get_layer_output([output])[0]
                get_layer_output = K.function([model.layers[3].input], [model.layers[3].output])
                output = get_layer_output([output])[0]
                index_of_max = np.argmax(output)
                print(f'output:{output} index_of_max:{index_of_max}')
                predictions.append(index_of_max)

            return predictions

        elif len(model.layers) == 5:
            predictions = []
            for index, row in X_test_m.iterrows():
                row_df = pd.DataFrame([row])
                row_df.columns = [f"x{i + 1}" for i in range(len(row_df.columns))]

                gmm_output = self.gmm_activation_smieja(row_df, gmm, model)

                input_data = np.array(gmm_output).reshape((1, len(gmm_output)))
                get_layer_output = K.function([model.layers[1].input], [model.layers[1].output])
                output = get_layer_output([input_data])[0]
                get_layer_output = K.function([model.layers[2].input], [model.layers[2].output])
                output = get_layer_output([output])[0]
                get_layer_output = K.function([model.layers[3].input], [model.layers[3].output])
                output = get_layer_output([output])[0]
                get_layer_output = K.function([model.layers[4].input], [model.layers[4].output])
                output = get_layer_output([output])[0]
                index_of_max = np.argmax(output)
                print(f'output:{output} index_of_max:{index_of_max}')
                predictions.append(index_of_max)

            return predictions

        else:
            print("Something wrong")

    def evaluate(self, y_test, y_gmm_nan):
        y_true = y_test
        y_pred = y_gmm_nan
        conf_matrix = confusion_matrix(y_true, y_pred)

        # Compute the accuracy
        accuracy = accuracy_score(y_true, y_pred)

        # Compute the precision
        precision_scores = precision_score(y_true, y_pred, labels=range(len(conf_matrix)), average=None)

        # Compute the recall
        recall = recall_score(y_true, y_pred, labels=range(len(conf_matrix)), average=None)

        # Compute the F1-score
        f1 = f1_score(y_true, y_pred, labels=range(len(conf_matrix)), average=None)

        # Compute the average accuracy
        avg_accuracy = sum(precision_scores) / 5

        # Compute the predicted probabilities
        y_scores = np.random.rand(len(y_true), len(np.unique(y_true)))

        # Compute the AUC for each class
        n_classes = conf_matrix.shape[0]
        auc_list = []
        for i in range(n_classes):
            auc_list.append(roc_auc_score(y_true == i, y_scores[:, i]))

        # Define the result dictionary
        result = {
            "confusion_matrix": conf_matrix,
            "accuracy": accuracy,
            "precision": precision_scores,
            "recall": recall,
            "f1_score": f1,
            "avg_accuracy": avg_accuracy,
            "auc": auc_list
        }
        return result

    def save_txt(self, filename, evaluate):
        with open(filename, 'w') as f:
            f.write(str(evaluate))
# Model_01-_03 Test
# p = 0.3,  0.6, 0,9

# List of models to test
models = [("Model_01", "predictions_D1_01_"), ("Model_02", "predictions_D1_02_"), ("Model_03", "predictions_D1_03_")]

# List of missing values percentages to test
p_values = [0.3, 0.6, 0.9]

for model_name, output_prefix in models:
    # Initialize model and prepare data
    model = gmm_model()
    X_raw, Y_raw, data_raw = model.import_data()
    X_train, X_test, y_train, y_test = model.train_test_split(X_raw, Y_raw, 0.33)

    # Load model and create gmm
    if model_name == "Model_01":
        trained_model = load_model('model_D1_01.h5')  # load the model instead of creating it
    elif model_name == "Model_02":
        trained_model = load_model('model_D1_02.h5')  # load the model instead of creating it
    elif model_name == "Model_03":
        trained_model = load_model('model_D1_03.h5')  # load the model instead of creating it

    gmm = model.gen_gmm(data_raw)

    for p in p_values:
        # Generate missing values data and predict values
        data_miss = model.gen_miss_values(X_test, p)
        predictions = model.predict_model(data_miss, trained_model)

        # Evaluate model and save results to file
        evaluate = model.evaluate(y_test, predictions)
        output_filename = output_prefix + str(p).replace(".", "_")
        model.save_txt(output_filename, evaluate)

output:[[0.00383312 0.01073705 0.02480408 0.60837466 0.35225114]] index_of_max:3
output:[[0.07791299 0.25838697 0.5583583  0.08190278 0.02343896]] index_of_max:2
output:[[0.01572244 0.00368973 0.05178486 0.45692235 0.4718807 ]] index_of_max:4
output:[[0.0026667  0.36658183 0.34199914 0.05414947 0.23460285]] index_of_max:1
output:[[1.4346615e-04 4.3685570e-20 6.3980857e-18 9.9985647e-01 4.1281396e-08]] index_of_max:3
output:[[1.0000000e+00 1.4494976e-35 4.3493465e-27 9.0406871e-10 1.5954300e-24]] index_of_max:0
output:[[0.00259048 0.12037914 0.23395067 0.2186268  0.42445296]] index_of_max:4
output:[[0.02164252 0.1192238  0.10762702 0.39200982 0.35949683]] index_of_max:3
output:[[0.00298232 0.44161087 0.29155666 0.10919348 0.15465659]] index_of_max:1
output:[[0.00257587 0.4854286  0.3754135  0.09919178 0.03739023]] index_of_max:1
output:[[9.9192315e-01 7.5632646e-03 5.1360199e-04 2.3666484e-20 3.3254672e-24]] index_of_max:0
output:[[0.0037173  0.10676705 0.06926247 0.30663776 0.51361537]

output:[[1.0000000e+00 2.8332907e-38 0.0000000e+00 1.2044910e-13 1.2839864e-11]] index_of_max:0
output:[[1.1567665e-02 5.3078870e-06 2.1761380e-05 9.6803856e-01 2.0366732e-02]] index_of_max:3
output:[[1.0867159e-02 4.8971469e-05 3.1021488e-04 9.0825534e-01 8.0518313e-02]] index_of_max:3
output:[[0.00106766 0.53161204 0.26818037 0.06355813 0.13558188]] index_of_max:1
output:[[0.00114567 0.20337369 0.6170753  0.08952266 0.08888265]] index_of_max:2
output:[[0.00704775 0.01222975 0.0013609  0.9257634  0.05359821]] index_of_max:3
output:[[0.00292942 0.50608134 0.21584767 0.05357262 0.22156899]] index_of_max:1
output:[[0.04958579 0.10952885 0.2262301  0.4152337  0.19942151]] index_of_max:3
output:[[3.03040966e-02 3.23155582e-05 2.77691051e-05 8.60885799e-01
  1.08749986e-01]] index_of_max:3
output:[[1.0000000e+00 2.0990737e-13 5.7492022e-16 4.5727376e-13 5.2014276e-23]] index_of_max:0
output:[[0.00199684 0.14385605 0.5660149  0.16437796 0.12375429]] index_of_max:2
output:[[0.03029151 0.00767

output:[[1.8085698e-02 4.0573370e-05 6.8379986e-06 9.5272768e-01 2.9139234e-02]] index_of_max:3
output:[[0.00849795 0.08590626 0.1955516  0.1875928  0.52245134]] index_of_max:4
output:[[0.04593849 0.69208676 0.22265461 0.02551052 0.01380955]] index_of_max:1
output:[[0.00197647 0.5004864  0.39335632 0.0122847  0.09189606]] index_of_max:1
output:[[7.6609850e-01 1.8100651e-03 2.2631535e-01 5.7071638e-03 6.8869900e-05]] index_of_max:0
output:[[0.00254867 0.2791342  0.21070854 0.32813224 0.17947637]] index_of_max:3
output:[[1.5338808e-03 3.6983645e-05 7.8154466e-04 7.1302807e-01 2.8461954e-01]] index_of_max:3
output:[[0.00077157 0.3638928  0.5342663  0.0201225  0.08094684]] index_of_max:2
output:[[0.00169857 0.07197033 0.37789068 0.3222156  0.22622477]] index_of_max:2
output:[[1.5081050e-03 6.7919143e-07 3.3845531e-06 9.7879899e-01 1.9688793e-02]] index_of_max:3
output:[[0.00288561 0.16173075 0.2351864  0.25826818 0.34192905]] index_of_max:4
output:[[0.0023062  0.11342827 0.5243862  0.19280

output:[[7.1042979e-01 6.9861929e-28 5.1530031e-29 2.8957018e-01 3.1483925e-15]] index_of_max:0
output:[[0.01180163 0.12852556 0.7402553  0.10475801 0.01465941]] index_of_max:2
output:[[0.06666087 0.12950355 0.46406078 0.11661235 0.22316244]] index_of_max:2
output:[[0.00832534 0.06895919 0.41804194 0.31452325 0.19015026]] index_of_max:2
output:[[1.0854808e-02 5.7634932e-04 4.1340301e-03 2.4192776e-01 7.4250704e-01]] index_of_max:4
output:[[0.10380429 0.06812731 0.49764    0.06898768 0.26144072]] index_of_max:2
output:[[0.00265721 0.01609808 0.08954103 0.53630537 0.35539818]] index_of_max:3
output:[[0.00103575 0.2966055  0.35904396 0.08362255 0.25969216]] index_of_max:2
output:[[0.00194621 0.34468463 0.44297534 0.07182264 0.1385712 ]] index_of_max:2
output:[[0.00376682 0.29507062 0.6206282  0.0159498  0.06458452]] index_of_max:2
output:[[0.00667311 0.3192171  0.5165891  0.07710356 0.08041718]] index_of_max:2
output:[[0.00311911 0.17044316 0.04145431 0.68109274 0.1038906 ]] index_of_max:

output:[[0.00449762 0.29691628 0.38647762 0.17497072 0.13713786]] index_of_max:2
output:[[0.00093346 0.17644255 0.36927006 0.04930946 0.40404442]] index_of_max:4
output:[[0.07803287 0.03569263 0.01125695 0.81865996 0.05635756]] index_of_max:3
output:[[0.00327057 0.4743509  0.3951374  0.07530869 0.05193244]] index_of_max:1
output:[[0.00206308 0.14805207 0.45073095 0.10020158 0.29895237]] index_of_max:2
output:[[6.1394105e-04 4.8587138e-20 2.3059078e-18 9.9938607e-01 5.0727838e-08]] index_of_max:3
output:[[0.0226008  0.01287393 0.02149615 0.43742746 0.5056017 ]] index_of_max:4
output:[[2.1979338e-03 1.4637753e-04 3.0831459e-05 8.0293530e-01 1.9468954e-01]] index_of_max:3
output:[[9.9989569e-01 3.0029698e-05 6.7135763e-05 2.5345557e-07 6.9713315e-06]] index_of_max:0
output:[[0.001635   0.23338228 0.3967813  0.10367236 0.26452914]] index_of_max:2
output:[[0.33114135 0.01424718 0.0047444  0.46168867 0.18817836]] index_of_max:3
output:[[0.00392006 0.21659076 0.3036735  0.21842968 0.25738603]

...

output:[[0.05093074 0.06669687 0.07722703 0.43408403 0.37106138]] index_of_max:3
output:[[0.03975958 0.10011811 0.14024808 0.1802456  0.5396286 ]] index_of_max:4
output:[[0.02392059 0.16433917 0.38889852 0.10148772 0.321354  ]] index_of_max:2
output:[[0.03116033 0.03834233 0.06508528 0.256051   0.609361  ]] index_of_max:4
output:[[0.06426752 0.09112933 0.07778353 0.52639645 0.24042319]] index_of_max:3
output:[[8.4638238e-02 1.7851251e-04 1.8455466e-04 9.1498959e-01 9.0277026e-06]] index_of_max:3
output:[[9.9186724e-01 2.4446023e-05 9.9148601e-05 2.5987154e-04 7.7492585e-03]] index_of_max:0
output:[[0.0591224  0.0244103  0.02833345 0.61354107 0.27459288]] index_of_max:3
output:[[9.9999535e-01 6.6218959e-08 1.6360921e-06 1.0482669e-06 1.9548786e-06]] index_of_max:0
output:[[0.04050779 0.0231698  0.01656104 0.35329455 0.56646675]] index_of_max:4
output:[[0.05134364 0.02381653 0.01679024 0.1618762  0.7461734 ]] index_of_max:4
output:[[0.04856514 0.07064828 0.04904225 0.50418645 0.3275579 ]

output:[[0.0331569  0.04070924 0.01725506 0.01581528 0.8930635 ]] index_of_max:4
output:[[0.13240144 0.00825601 0.01493108 0.6849314  0.15948017]] index_of_max:3
output:[[0.01543242 0.17188123 0.21760361 0.3798732  0.21520956]] index_of_max:3
output:[[0.08005233 0.08033247 0.07519998 0.35756993 0.40684533]] index_of_max:4
output:[[0.04054103 0.03206411 0.04270128 0.3620155  0.5226781 ]] index_of_max:4
output:[[0.04560542 0.01642441 0.01432944 0.44129503 0.48234564]] index_of_max:4
output:[[0.32585296 0.02464141 0.27230912 0.07100477 0.30619174]] index_of_max:0
output:[[0.2878348  0.02868223 0.04383378 0.30311427 0.33653492]] index_of_max:4
output:[[0.04912554 0.0099636  0.01027992 0.27824324 0.6523877 ]] index_of_max:4
output:[[0.9129695  0.01960984 0.00215693 0.00994347 0.05532027]] index_of_max:0
output:[[0.04430816 0.08593771 0.06667583 0.27085057 0.53222775]] index_of_max:4
output:[[0.02896059 0.06250567 0.00624274 0.26467514 0.63761586]] index_of_max:4
output:[[0.06694586 0.053565

output:[[0.04485003 0.06702863 0.21783464 0.23766302 0.4326236 ]] index_of_max:4
output:[[9.8415655e-01 3.1944086e-07 3.5955952e-06 3.1026742e-05 1.5808441e-02]] index_of_max:0
output:[[0.077375   0.10245645 0.09043805 0.4710174  0.25871313]] index_of_max:3
output:[[0.07380893 0.01914624 0.0165816  0.40650743 0.48395574]] index_of_max:4
output:[[0.06487352 0.0742249  0.09410862 0.29896995 0.46782306]] index_of_max:4
output:[[0.03569853 0.09009288 0.08628743 0.2753458  0.5125753 ]] index_of_max:4
output:[[0.05718013 0.17958839 0.18081625 0.25941494 0.32300022]] index_of_max:4
output:[[2.3294832e-01 6.4266339e-02 2.9638296e-04 9.5579699e-03 6.9293100e-01]] index_of_max:4
output:[[0.02692131 0.15313114 0.10174005 0.19010617 0.52810127]] index_of_max:4
output:[[0.07901783 0.00716885 0.08497012 0.76379335 0.06504985]] index_of_max:3
output:[[0.07132006 0.12462546 0.16202323 0.33885947 0.30317166]] index_of_max:3
output:[[0.05657797 0.18008758 0.0721648  0.04034616 0.6508235 ]] index_of_max:

output:[[0.027075   0.02418799 0.06449847 0.5491472  0.3350914 ]] index_of_max:3
output:[[9.3415010e-01 1.6594250e-14 4.0156864e-11 6.5547489e-02 3.0242570e-04]] index_of_max:0
output:[[0.04217567 0.07411133 0.65162617 0.11363303 0.11845373]] index_of_max:2
output:[[0.05549751 0.08316899 0.1944108  0.20556192 0.4613607 ]] index_of_max:4
output:[[0.54694796 0.03297389 0.2582176  0.14831449 0.01354611]] index_of_max:0
output:[[0.05869167 0.05819135 0.05839462 0.3100766  0.51464576]] index_of_max:4
output:[[0.04791121 0.15678045 0.07814174 0.39294186 0.32422474]] index_of_max:3
output:[[0.02327565 0.0245038  0.01067063 0.39523125 0.54631865]] index_of_max:4
output:[[0.036532   0.09537915 0.09331872 0.17042583 0.60434425]] index_of_max:4
output:[[9.0352452e-01 7.6592689e-05 4.1224313e-04 9.4580188e-02 1.4064180e-03]] index_of_max:0
output:[[0.05214177 0.07791173 0.06824455 0.3229829  0.47871903]] index_of_max:4
output:[[0.04561037 0.0655865  0.0592845  0.3585761  0.47094253]] index_of_max:

output:[[2.3182489e-03 1.0065074e-06 2.8884921e-07 9.9667007e-01 1.0104771e-03]] index_of_max:3
output:[[0.10389856 0.13049349 0.11342535 0.15048969 0.50169295]] index_of_max:4
output:[[0.05938066 0.14340253 0.15400949 0.11708391 0.5261234 ]] index_of_max:4
output:[[0.04960198 0.21738894 0.10498495 0.21898915 0.40903497]] index_of_max:4
output:[[0.03953033 0.07370792 0.07423911 0.31021664 0.502306  ]] index_of_max:4
output:[[0.05596031 0.04592512 0.10257407 0.20128566 0.59425485]] index_of_max:4
output:[[0.5681226  0.06872491 0.14661114 0.19260229 0.02393903]] index_of_max:0
output:[[9.5630800e-03 1.2400200e-03 7.6074403e-04 1.3377438e-01 8.5466176e-01]] index_of_max:4
output:[[0.05795461 0.12716617 0.13318434 0.18758006 0.49411482]] index_of_max:4
output:[[0.03130568 0.10506894 0.0838251  0.35465372 0.42514658]] index_of_max:4
output:[[0.02469756 0.28127155 0.21297134 0.20481846 0.27624115]] index_of_max:1
output:[[0.06936362 0.08333126 0.06650129 0.32412842 0.45667544]] index_of_max: